<a href="https://colab.research.google.com/github/brunafrade/2StepsLearning/blob/main/Notebook/train_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install keras==2.0
#!pip install tensorflow==1.15.0
import tensorflow as tf
import matplotlib.pyplot as plt
plt.switch_backend('agg')
from tensorflow import keras
from keras import backend as k
from keras.models import Model
from keras.layers import Input, Flatten,GlobalAveragePooling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D, UpSampling2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.core import Activation, Dropout, Dense, Reshape

# from keras.layers.normalization import BatchNormalization
from keras.layers.normalization.batch_normalization import BatchNormalization
# from tensorflow.keras.layers import BatchNormalization
from keras.optimizers import gradient_descent_v2,adam_v2,rmsprop_v2
# from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.constraints import maxnorm
from keras import optimizers
import os
import numpy as np
import time


from PIL import Image, ImageDraw
import numpy as np
from keras import backend as K
import os
from pandas.io.parsers import read_csv
import os
import gzip
import pickle
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
plt.switch_backend('agg')

import numpy as np
import gzip
from keras.utils import np_utils
import socket
from sklearn.model_selection import KFold

from google.colab import drive
drive.mount("/drive")

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [ ]:

import tensorflow as tf

def get_session(gpu_fraction=0.2):
    '''Assume that you have 6GB of GPU memory and want to allocate ~2GB'''

    num_threads = os.environ.get('OMP_NUM_THREADS')
    gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=gpu_fraction)

    if num_threads:
        return tf.Session(config=tf.ConfigProto(
            gpu_options=gpu_options, intra_op_parallelism_threads=num_threads))
    else:
        return tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

In [ ]:
def create_folder(directory):
	if not os.path.exists(directory):
		os.makedirs(directory)
def cross_validation(X_train,Y_train,train_index,test_index):
    # kf = KFold(n_splits=5)
    # fold=0
    # for (train_index, test_index) in kf.split(X_train):
    X_src_tr= np.zeros([train_index.shape[0],32,32,1])
    X_src_tes= np.zeros([test_index.shape[0],32,32,1])
    y_src_tr= np.zeros([train_index.shape[0],6])
    y_src_tes= np.zeros([test_index.shape[0],6])


    j=0
    for index_train in train_index:

        X_src_tr[j] = X_train[index_train]
        y_src_tr[j] = Y_train[index_train]
        
        j+=1

    j=0
    for index_test in  test_index:

        X_src_tes[j] = X_train[index_test]
        y_src_tes[j] = Y_train[index_test]
        
        j+=1

    return X_src_tr,y_src_tr,X_src_tes,y_src_tes

def preprocess_images(X, tmin=-1, tmax=1):
	V = X * (tmax - tmin) / 255.
	V += tmin
	return V

def postprocess_images(V, omin=-1, omax=1):
	X = V - omin
	X = X * 255. / (omax - omin)
	return X

def show_images(Xo, padsize=1, padval=0, filename=None, title=None):
	# data format : channel_first
	X = np.copy(Xo)
	[n, d1, d2, c] = X.shape
	if c== 1:
		X = np.reshape(X, (n, d1, d2))

	n = int(np.ceil(np.sqrt(X.shape[0])))
	
	padding = ((0, n ** 2 - X.shape[0]), (0, padsize), (0, padsize)) + ((0, 0), ) * (X.ndim - 3)
	canvas = np.pad(X, padding, mode='constant', constant_values=(padval, padval))

	canvas = canvas.reshape((n, n) + canvas.shape[1:]).transpose((0, 2, 1, 3) + tuple(range(4, canvas.ndim + 1)))
	canvas = canvas.reshape((n * canvas.shape[1], n * canvas.shape[3]) + canvas.shape[4:])

	if title is not None:
		title_canv = np.zeros((50, canvas.shape[1]))
		title_canv = title_canv.astype('uint8')
		canvas = np.vstack((title_canv, canvas)).astype('uint8')
		
		I = Image.fromarray(canvas)
		d = ImageDraw.Draw(I)
		fill = 255
		d.text((10, 10), title, fill=fill, font=fnt)
	else:
		canvas = canvas.astype('uint8')
		I = Image.fromarray(canvas)

	if filename is None:
		I.show()
	else:
		I.save(filename)

	return I
def read_dhiego_dataset(FTRAIN_source):


		df_train_src = read_csv(os.path.expanduser(FTRAIN_source))  # load pandas dataframe
	 	
		df_train_src['image'] = df_train_src['image'].apply(lambda im: np.fromstring(im, sep=' '))
		
		df_train_src = df_train_src.dropna()  # drop all rows that have missing values in them
		
		
		X_train_src = np.vstack(df_train_src['image'].values)   # scale pixel values to [0, 1]
		X_train_src = X_train_src.astype(np.uint8)

		
		y_train_src = df_train_src[df_train_src.columns[:-1]].values
		y_train_src = y_train_src.astype(np.uint8)

		y_train_src=y_train_src/32.

		X_train_src = X_train_src.reshape(X_train_src.shape[0], 32, 32, 1)

		

		return X_train_src,y_train_src
def load(FTRAIN_source):
    df_train_src = read_csv(os.path.expanduser(FTRAIN_source))  # load pandas dataframe

    # The Image column has pixel values separated by space; convert
    # the values to numpy arrays:
    df_train_src['image'] = df_train_src['image'].apply(lambda im: np.fromstring(im, sep=' '))
    
    print('source ',df_train_src.count())  # prints the number of values for each column

    df_train_src = df_train_src.dropna()  # drop all rows that have missing values in them

    X_train_src = np.vstack(df_train_src['image'].values)/255.   # scale pixel values to [0, 1]

    X_train_src = X_train_src.astype('float32')


    df_train_src['image_gt'] = df_train_src['image_gt'].apply(lambda im: np.fromstring(im, sep=' '))
    
    print('source ',df_train_src.count())  # prints the number of values for each column

    df_train_src = df_train_src.dropna()  # drop all rows that have missing values in them

    gt_train_src = np.vstack(df_train_src['image_gt'].values)/255   # scale pixel values to [0, 1]
    # gt_train_src = gt_train_src.astype('float32')

    y_train_src = (df_train_src[df_train_src.columns[:-2]].values )

   
    
   
    # y_train_src = y_train_src.astype('uint8')
    # idx10 = np.where(y_train_src > 32)
    # y_train_src[idx10] = 1
    # print y_train_src
    #y_train_src=y_train_src/32.
    y_train_src=(y_train_src-16)/16.

    X_train_src = X_train_src.reshape(X_train_src.shape[0], 32, 32, 1)
    gt_train_src = gt_train_src.reshape(gt_train_src.shape[0], 32, 32, 3)
    # y_train_src = y_train_src.reshape(y_train_src.shape[0],1,1,6)
    y_train_src = y_train_src.reshape(y_train_src.shape[0],6)
    return X_train_src,gt_train_src,y_train_src

def get_impulse_noise(X, level):
	p = 1. - level
	Y = X * np.random.binomial(1, p, size=X.shape)
	return Y

def get_gaussian_noise(X, std):
	# X: [n, c, d1, d2] images in [0, 1]
	Y = np.random.normal(X, scale=std)
	Y = np.clip(Y, 0., 1.)
	return Y	

def get_flipped_pixels(X):
	# X: [n, c, d1, d2] images in [0, 1]
	Y = 1. - X
	Y = np.clip(Y, 0., 1.)
	return Y


def iterate_minibatches(inputs, targets, batchsize=128, shuffle=True):

	assert len(inputs) == len(targets)

	if shuffle:
		indices = np.arange(len(inputs))
		np.random.shuffle(indices)

	for start_idx in range(0, len(inputs), batchsize):
		end_idx = start_idx + batchsize
		if end_idx > len(inputs):
			end_idx = start_idx + (len(inputs) % batchsize)

		if shuffle:
			excerpt = indices[start_idx:end_idx]
	
		else:
			excerpt = slice(start_idx, end_idx)

		yield (inputs[excerpt], targets[excerpt])



def save_weights(model, PARAMDIR, CONF):
	# model: keras model
	print(' == save weights == ')
	print (PARAMDIR)

	# save weights
	PARAMPATH = os.path.join(PARAMDIR, '%s_weights.h5') % CONF
	model.save(PARAMPATH)
	
	# save architecture
	CONFPATH = os.path.join(PARAMDIR, '%s_conf.json') % CONF
	archjson = bytes(model.to_json(), 'utf-8')

	open(CONFPATH, 'wb').write(archjson)


def clip_relu(x):
	y = K.maximum(x, 0)
	return K.minimum(y, 1)



# def plot_sample(x, y,axis):
# 	# y=y.reshape(6)
# 	img = x.reshape(32, 32)
# 	axis.imshow(img,cmap='gray')
# 	axis.scatter(y[0]*32 , y[1]*32 , marker='<', s=5)
# 	axis.scatter(y[2]*32 , y[3]*32 , marker='>', s=5)
# 	axis.scatter(y[4]*32 , y[5]*32 , marker='o', s=5)

	#axis.scatter(y[6]*32 , y[7]*32 , marker='o', s=5)
	#axis.scatter(y[8]*32 , y[9]*32 , marker='o', s=5)
	#axis.scatter(y[10]*32 , y[11]*32 , marker='o', s=5)
	#axis.scatter(y[12]*32 , y[13]*32 , marker='o', s=5)
	#axis.scatter(y[14]*32 , y[15]*32 , marker='o', s=5)
	#axis.scatter(y[16]*32 , y[17]*32 , marker='o', s=5)
	#axis.scatter(y[18]*32 , y[19]*32 , marker='o', s=5)
def plot_sample(x, y,axis):
	#y=y.reshape(6)
	img = x.reshape(32, 32)
	axis.imshow(img,cmap='gray')
	#axis.scatter(y[0]*16.+16 , y[1]*16.+16 , marker='<', s=15)
	#axis.scatter(y[2]*16.+16 , y[3]*16.+16 , marker='>', s=15)
	#axis.scatter(y[4]*16.+16 , y[5]*16.+16 , marker='o', s=15)
def visualization (model, x_heat,x,PARAMDIR,image_name):
# visualization 
		
	Xs = x_heat[:36]
	# Xs = x
	# print Xs
	# Xs = postprocess_images(Xs, omin=0, omax=1)
	# Xs = np.reshape(Xs, (len(Xs), Xs.shape[3], Xs.shape[1], Xs.shape[2]))
	# #show_images(Xs, filename=image_name)				
	# Xs_pred = drcn.convae_model.predict(Xs)
	# Xs_pred = postprocess_images(Xs_pred, omin=0, omax=1)
	# Xs_pred = np.reshape(Xs_pred, (len(Xs_pred), Xs_pred.shape[3], Xs_pred.shape[1], Xs_pred.shape[2]))
	# imgfile = PARAMDIR+'saida_autoencoder/o_'+image_name
	# show_images(Xs, filename=imgfile)
	imgfile = PARAMDIR+'/saida_autoencoder/'+image_name

	# show_images(Xs_pred, filename=imgfile)

	Xz = model.predict(x_heat[:36])
	Xz = postprocess_images(Xz, omin=0, omax=1)
	Xz = np.reshape(Xz, (len(Xz), Xz.shape[3], Xz.shape[1], Xz.shape[2]))
	fig = plt.figure(figsize=(1, 1))
	fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)
	for i in range(Xs.shape[0]):
		ax = fig.add_subplot(np.sqrt(Xs.shape[0]), np.sqrt(Xs.shape[0]), i + 1, xticks=[], yticks=[])
		plot_sample(Xz[i],[0], ax)
	
	fig.savefig(imgfile)

	plt.close()

def new_accuracy(keypoint_pred, keypoint_gt):
    keypoint_pred=keypoint_pred.reshape(keypoint_pred.shape[0],6)
    keypoint_gt=keypoint_gt.reshape(keypoint_gt.shape[0],6)
    # Tamanhos incompativeis
    if keypoint_pred.shape != keypoint_gt.shape:
        return

    # Coordenadas faltando
    if (keypoint_pred.shape[1] % 2 != 0) | (keypoint_gt.shape[1] % 2 != 0):
        return
    # print keypoint_pred
    THRESHOLD = 0.1
    FACE_SIZE = 2
    FAILURE_COUNT = 0
    euc_dist = np.zeros((keypoint_gt.shape[0], int(keypoint_gt.shape[1]/2)))

    # Supondo que os pontos estao ordenados em X, Y, X, Y, X, Y...
    for i in range(int(keypoint_gt.shape[1]/2)):
        euc_dist[:, i] = np.sqrt((keypoint_pred[:, 2*i] - keypoint_gt[:, 2*i])**2 + (keypoint_pred[:, 2*i + 1] - keypoint_gt[:, 2*i + 1])**2)
        
        for j in range(keypoint_gt.shape[0]):
            if euc_dist[j, i] > (THRESHOLD * FACE_SIZE):
                FAILURE_COUNT += 1

    AVG_FAILURE_RATE = float(FAILURE_COUNT) / (keypoint_gt.shape[0] * keypoint_gt.shape[1]/2)

    return 1 -AVG_FAILURE_RATE  


In [ ]:
def plot_grafic(gen_loss,losses,val_losses,test_losses,epoca, dire):
	

	
	plt.plot( epoca, gen_loss, 'k:',label='rec_cat', color='orange') # linha pontilha orange

	plt.plot( epoca, losses, 'k:',label='trei_detec_hum', color='blue')  # linha tracejada azul
	
	plt.plot( epoca, val_losses, 'k--',label='detec_hum', color='black')  # linha tracejada azul
	
	plt.plot( epoca, test_losses, 'k--',label='detec_cat', color='green')  # linha tracejada azul

	plt.axis([0, 500, 0, 0.2])
	plt.title("Autoencoder:treino")
	plt.legend()
	plt.grid(True)
	plt.xlabel("Epoca")
	plt.ylabel("Erro")
	plt.savefig(dire+"/grafic_{}.png".format(len(epoca)))
	plt.close()




class MODEL(object):
	def __init__(self, name='svhn-mnist'):
		"""
		Class constructor
		"""
		self.name = name

	def extract_feat(self, _input, dense_dim=1000, dy=10, nb_filters=[64, 128], kernel_size=(3, 3), pool_size=(2, 2), 
			dropout=0.5, bn=True, output_activation='softmax', opt='adam',trainable=True):

			_h = _input

			
			# _h = Conv2D(24, (5, 5), border_mode='same', init='he_normal',activation='relu')(_h)
			i=0

			for i, nf in enumerate(nb_filters):
				name='conv%s'%i

				_h = Conv2D(nf, kernel_size, padding='same',activation='relu')(_h)	
				
				if bn:
					_h = BatchNormalization()(_h)
				if i<2:
					_h = MaxPooling2D(pool_size=pool_size, padding='same',trainable=trainable)(_h)
				i+=1
				# kernel_size=(3,3)
				

			_f =_h

			_h = Flatten()(_h)
			# _h = Dropout(dropout)(_h)
			
			_h = Dense(1024,activation='relu',trainable=trainable)(_h)
			_h = Dropout(dropout)(_h)
			_h = Dense(1024,activation='relu',trainable=trainable)(_h)
			_h = Dropout(dropout)(_h)
			
			return _h,_f
			
	def create_model(self, input_shape=(1, 32, 32), dense_dim=1000, dy=10, nb_filters=[64, 128], kernel_size=(3, 3), pool_size=(2, 2), 
			dropout=0.5, bn=True, output_activation='softmax', opt='adam'):	

			[d1, d2, c] = input_shape

			if opt == 'adam':
				opt = tf.keras.optimizers.Adam(lr=3e-4)
			elif opt == 'rmsprop':
				#opt = RMSprop(lr=1e-4)
				opt = tf.keras.optimizers.RMSprop(learning_rate=1e-4)


			#convnet e encoder	
			_input = Input(shape=input_shape)

			_h,_f = self.extract_feat(_input,dense_dim,dy,nb_filters,kernel_size,pool_size,dropout,bn,output_activation,opt,trainable=True)
			
			_y1 = Dense(dy,activation='tanh',trainable=True)(_h)
			
			# _h,_f = self.extract_feat(_input,dense_dim,dy,nb_filters,kernel_size,pool_size,dropout,bn,output_activation,opt,trainable=True)

			[_,wflat, hflat, cflat]=_f.get_shape().as_list()
			#decoder
			_h = Dense (1024)(_h)
			_h = Dense (wflat* hflat* cflat)(_h)
			_h = Reshape((wflat, hflat, nb_filters[-1]))(_h)
			pool_size=(1,1)
			i=0
			for nf in reversed(nb_filters):


				_h = tf.keras.layers.Conv2DTranspose(nf, kernel_size, strides=pool_size, padding='same',activation='relu')(_h)
				if bn:
					_h = BatchNormalization()(_h)

				if i>0:
					pool_size=(2,2)
				i+=1
				

			_h = Conv2D(1, kernel_size, padding='same', activation='relu')(_h)



			sgd = tf.keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
			self.convnet_model = Model(_input, _y1)
			self.convnet_model.compile(loss='mean_absolute_error', optimizer=tf.keras.optimizers.Adam(lr=3e-4))	

			self.convae_model = Model(_input, _h)	
			self.convae_model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(lr=3e-4))
			print(self.convnet_model.summary())
			print(self.convae_model.summary())



	


	###  just in case want to run convnet and convae separately, below are the training modules  ###
###  just in case want to run convnet and convae separately, below are the training modules  ###
	def fit_model(self, X, Y, Xu,supervised=None, nb_epoch=50, batch_size=128, shuffle=True,
			validation_data=None, test_data=None, PARAMDIR=None, CONF=None, tresh='none'):

	
	
		history = {}
		history['losses'] = []
		history['accs'] = []
		history['gen_losses'] = []
		history['val_losses'] = []
		history['val_accs'] = []
		history['test_losses'] = []
		history['test_accs'] = []
		history['elapsed_times'] = []

		best_ep = 1
		
		

		gdatagen = ImageDataGenerator(
			    featurewise_center=False, # set input mean to 0 over the dataset
			    samplewise_center=False, # set each sample mean to 0
			    featurewise_std_normalization=False, # divide inputs by std of the dataset
			    samplewise_std_normalization=False, # divide each input by its std
			    zca_whitening=False, # apply ZCA whitening
			    rotation_range=0.9, # randomly rotate images in the range (degrees, 0 to 180)
			    width_shift_range=0.4, # randomly shift images horizontally (fraction of total width)
			    height_shift_range=0.4, # randomly shift images vertically (fraction of total height)
			    horizontal_flip=0.4, # randomly flip images
			    vertical_flip=0.4
			) # randomly flip images
		ep=[]
		control=11
		for e in range(nb_epoch):
			ep.append(e)
			start_t = time.time()
			

			## convnet training
			loss = 0.
			gen_loss = 0.
			loss_cat = 0.
			n_batch = 0
			(X_, Y_) = supervised

			total_batches = X.shape[0] / batch_size
			acc=-1

			for (X_batch_, Y_batch_),(X_batch, Y_batch), (Xu_batch,Yu_batch) in zip(iterate_minibatches(X_, Y_, batchsize=batch_size, shuffle=shuffle),iterate_minibatches(X, Y, batchsize=batch_size, shuffle=shuffle),gdatagen.flow(Xu, np.copy(Xu), batch_size=batch_size, shuffle=shuffle)):				
				Xu_batch = get_impulse_noise(Xu_batch, 0.5)
				print( X_batch.shape[0], " ",Xu_batch.shape[0], " ",X_batch_.shape[0])
				l = self.convae_model.train_on_batch(Xu_batch, Yu_batch)
				gen_loss += l
				
				l = self.convnet_model.train_on_batch(X_batch, Y_batch)
				loss += l
				if X_.shape[0] != 0:
					l = self.convnet_model.train_on_batch(X_batch_, Y_batch_)
					loss_cat += l


				n_batch += 1
				if n_batch >= total_batches:
					break


				

			loss /= n_batch
			history['losses'].append(loss)
			
			gen_loss /= n_batch
			history['gen_losses'].append(gen_loss)
						
			elapsed_t = time.time() - start_t
			history['elapsed_times'].append(elapsed_t)

						
			val_loss = -1
			val_acc = -1
			best_val_acc = -1
			if validation_data is not None:
				(X_val, Y_val) = validation_data
				val_loss = 0.
				n_batch = 0
				for Xv, Yv in iterate_minibatches(X_val, Y_val ,batch_size, shuffle=False):
					l = self.convnet_model.test_on_batch(Xv, Yv)
					val_loss += l
					n_batch += 1
				val_loss /= n_batch
				history['val_losses'].append(val_loss)

			
			test_loss = -1
			test_acc = -1
			if test_data is not None:
				(X_test, Y_test) = test_data
				test_loss = 0.
				n_batch = 0
				for Xt, Yt in iterate_minibatches(X_test, Y_test, batch_size, shuffle=False):
					l = self.convnet_model.test_on_batch(Xt, Yt)
					test_loss += l
					n_batch += 1

				test_loss /= n_batch

			print('Epoch-%d: (loss: %.3f, gen_loss: %.3f), (val_loss: %.3f), (test_Loss: %.3f)  -- %.2f sec' % \
				((e+1), loss, gen_loss, val_loss, test_loss,  elapsed_t))


			control+=1
			if control>=10:
				control=0

				acc = new_accuracy(self.convnet_model.predict(X), Y)

				val_acc = new_accuracy(self.convnet_model.predict(X_val), Y_val)

				test_acc = new_accuracy(self.convnet_model.predict(X_test), Y_test)
				save_weights(self.convnet_model, PARAMDIR,CONF)
				#save_weights(self.convae_model, CONF, 'ours_reconstrutor_{}'.format(tresh))

				print('Epoch-%d: (loss: %.3f, acc: %.3f, gen_loss: %.3f), (val_loss: %.3f, val_acc: %.3f), (test_Loss: %.3f, test_acc: %.3f)  -- %.2f sec' % \
				((e+1), loss, acc, gen_loss, val_loss, val_acc, test_loss, test_acc,  elapsed_t))

In [ ]:

 # KTF.set_session(get_session())
# Load datasets
animal=['horse','human','dog']
print('Load datasets')
dir_source = '/drive/My Drive/Dataset_mestrado/training_human_32x32.csv'
X_source, _, Y_source = load(dir_source)
print (X_source.shape)
for name_animal in animal:

        name_animal='cat'
        dir_target='/drive/My Drive/Dataset_mestrado/training_cat_32x32.csv'
        X_target, _, Y_target = load(dir_target)
        PARAMDIR='/drive/My Drive/Dataset_mestrado/weights/'+name_animal+'/'
        create_folder(PARAMDIR)
        param=[0,10,50,100,200,500,1000]


        for tresh in param:
            k.set_session(get_session())
            CONF='ours_horse_tresh_'+str(tresh)
            print('Create Model')
            model = MODEL()
            nb_classes=6#nb_filters=[300,250,200,150]
            input_shape = (X_source.shape[1], X_source.shape[2], X_source.shape[3])
            print (input_shape)
            model.create_model(input_shape=input_shape, dense_dim=500, dy=nb_classes, nb_filters=[300,200,150,100], kernel_size=(3,3), pool_size=(2, 2), 
                    dropout=0.5, bn=False, output_activation='softmax', opt='rmsprop')
            #tresh=100
            print('Train supervised...')
            print ("name_animal ",name_animal)

            print( X_target[:tresh].shape)
            if tresh ==0:
            	supervised=(X_target[:1], Y_target[:1])
            else:
            	supervised=(X_target[:tresh], Y_target[:tresh])

            model.fit_model(X_source, Y_source, X_target,supervised=supervised, validation_data=(X_source,Y_source), 
                    test_data=(X_target[1000:],Y_target[1000:]),
                    nb_epoch=200, batch_size=500,
                    PARAMDIR=PARAMDIR, CONF=CONF,tresh=tresh
            )
            del model
            K.clear_session()
			

Load datasets
source  left_eye_center_x     8560
left_eye_center_y     8560
right_eye_center_x    8560
right_eye_center_y    8560
nose_tip_x            8560
nose_tip_y            8560
image                 8560
image_gt              8560
dtype: int64
source  left_eye_center_x     8560
left_eye_center_y     8560
right_eye_center_x    8560
right_eye_center_y    8560
nose_tip_x            8560
nose_tip_y            8560
image                 8560
image_gt              8560
dtype: int64
(8560, 32, 32, 1)
source  left_eye_center_x     9846
left_eye_center_y     9846
right_eye_center_x    9846
right_eye_center_y    9846
nose_tip_x            9846
nose_tip_y            9846
image                 9846
image_gt              9846
dtype: int64
source  left_eye_center_x     9846
left_eye_center_y     9846
right_eye_center_x    9846
right_eye_center_y    9846
nose_tip_x            9846
nose_tip_y            9846
image                 9846
image_gt              9846
dtype: int64
Create Model
(32, 32

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


None
Train supervised...
name_animal  cat
(0, 32, 32, 1)
500   500   1
Epoch-1: (loss: 0.346, gen_loss: 0.242), (val_loss: 0.231), (test_Loss: 0.196)  -- 47.09 sec
 == save weights == 
/drive/My Drive/Dataset_mestrado/weights/cat/
Epoch-1: (loss: 0.346, acc: 0.424, gen_loss: 0.242), (val_loss: 0.231, val_acc: 0.424), (test_Loss: 0.196, test_acc: 0.238)  -- 47.09 sec
500   500   1
Epoch-2: (loss: 0.353, gen_loss: 0.247), (val_loss: 0.233), (test_Loss: 0.211)  -- 42.28 sec
500   500   1
Epoch-3: (loss: 0.378, gen_loss: 0.233), (val_loss: 0.234), (test_Loss: 0.252)  -- 40.67 sec
500   500   1
Epoch-4: (loss: 0.363, gen_loss: 0.200), (val_loss: 0.244), (test_Loss: 0.283)  -- 41.90 sec
500   500   1
Epoch-5: (loss: 0.373, gen_loss: 0.136), (val_loss: 0.239), (test_Loss: 0.281)  -- 43.50 sec
500   500   1
Epoch-6: (loss: 0.353, gen_loss: 0.070), (val_loss: 0.222), (test_Loss: 0.257)  -- 40.50 sec
500   500   1
Epoch-7: (loss: 0.277, gen_loss: 0.190), (val_loss: 0.267), (test_Loss: 0.281)  --

KeyboardInterrupt: ignored